# 14. Advanced Panel Data Methods

## 14.1. Fixed Effects Estimation

$y_{it} = \beta_0 + \beta_1x_{it1} + . . .  + \beta_kx_{itk} + a_i + u_{it}$

$\bar{y_i} = \beta_0 + \beta_1\bar{x}_{i1} + . . . + \beta_k\bar{x}_{ik} + a_i + \bar{u_i}$

$\ddot{y} = y_{it} - \bar{y_i} $

$\ddot{y} = \beta_1\ddot{x}_{it1} + . . . +  \beta_k\ddot{x}_{itk} + \ddot{u}_{it}$


### Exapmle 14.2: Has the Return to Education Changed over Time

In [1]:
import wooldridge as woo
import pandas as pd
import statsmodels.formula.api as smf
import linearmodels as plm
import scipy.stats as stats
from supplementaryFunctions import *

In [2]:
def LM_summary(results, round_dig = 4):
    summary = {"$\\beta $":results.params, 
           "$t$": results.tstats,
           "$$P>|t|$$":results.pvalues,
           # calculate standard errors by taking the square root of the variance values 
           # along the diagonal of the covariance matrix 
          "$SE$":results.std_errors}
    summary = pd.DataFrame(summary)
    # add r^2 using index name
    summary.index.name = "$$r^2: "+str(round(results.rsquared,round_dig)) + "$$"
    return summary

#### The authors use *drop_absorbed* quite freely. This is problematic since it makes us lazy in identifying source of multicolinearity.

In [3]:
wagepan = woo.dataWoo("wagepan")
wagepan.set_index(["nr","year"], drop = False, inplace = True)
formula = "lwage ~ married + union + C(year)*educ + EntityEffects"
reg = plm.PanelOLS.from_formula(formula = formula,
                               data = wagepan, drop_absorbed=True)
results = reg.fit()
LM_summary(results)

C:\Users\JLCat\AppData\Local\Temp\ipykernel_21720\2493808540.py:6: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

educ

  results = reg.fit()


,$\beta $,$t$,$$P>|t|$$,$SE$
$$r^2: 0.1708$$,,,,
C(year)[T.1980],1.362459,83.903110,0.000000,0.016238
C(year)[T.1981],1.340043,9.230656,0.000000,0.145173
C(year)[T.1982],1.356698,9.348067,0.000000,0.145131
C(year)[T.1983],1.372888,9.456084,0.000000,0.145186
C(year)[T.1984],1.446833,9.961677,0.000000,0.145240
C(year)[T.1985],1.412184,9.731511,0.000000,0.145115
C(year)[T.1986],1.428065,9.840444,0.000000,0.145122
C(year)[T.1987],1.452904,10.006096,0.000000,0.145202
married,0.054820,2.977340,0.002926,0.018413


In [4]:
# names = ["lwage", "married","union" , "educ"]
# X, y = build_X_y_matrices(wagepan, names, log_vars = None)#, constant = True)

# reg = plm.PanelOLS(y, X, entity_effects = True, time_effects = True, drop_absorbed = True)
# results = reg.fit()
# LM_summary(results)

## 14.2. Random Effects Models

$\dot{y}_{it} = y_{it} - \theta \bar{y}_i$

$\dot{y}_{it} = \beta_0(1-\theta) + \beta_1\dot{x}_{it1} + . . . + \beta_k\dot{x}_{itk} + \dot{\nu}_{it}$

$\nu_{it} = a_i + u_{it}$

$\theta = 1 - \sqrt{\frac{\theta_u^2}{\theta_u^2 + T\theta_a^2}}$

In [5]:
wagepan = woo.dataWoo("wagepan")
wagepan.groupby("nr").var()

,year,agric,black,bus,construc,ent,exper,fin,hisp,poorhlth,...,union,lwage,d81,d82,d83,d84,d85,d86,d87,expersq
nr,,,,,,,,,,,,,,,,,,,,,
13,6.0,0.0,0.0,0.214286,0.000000,0.000,6.0,0.000000,0.0,0.0,...,0.125000,0.681418,0.125,0.125,0.125,0.125,0.125,0.125,0.125,510.0
17,6.0,0.0,0.0,0.000000,0.267857,0.000,6.0,0.000000,0.0,0.0,...,0.000000,0.009753,0.125,0.125,0.125,0.125,0.125,0.125,0.125,1374.0
18,6.0,0.0,0.0,0.000000,0.000000,0.000,6.0,0.000000,0.0,0.0,...,0.000000,0.186106,0.125,0.125,0.125,0.125,0.125,0.125,0.125,1374.0
45,6.0,0.0,0.0,0.125000,0.125000,0.000,6.0,0.000000,0.0,0.0,...,0.214286,0.049972,0.125,0.125,0.125,0.125,0.125,0.125,0.125,750.0
110,6.0,0.0,0.0,0.125000,0.000000,0.000,6.0,0.214286,0.0,0.0,...,0.125000,0.009836,0.125,0.125,0.125,0.125,0.125,0.125,0.125,1758.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12451,6.0,0.0,0.0,0.000000,0.000000,0.125,6.0,0.000000,0.0,0.0,...,0.000000,0.137637,0.125,0.125,0.125,0.125,0.125,0.125,0.125,750.0
12477,6.0,0.0,0.0,0.000000,0.000000,0.000,6.0,0.000000,0.0,0.0,...,0.000000,0.001519,0.125,0.125,0.125,0.125,0.125,0.125,0.125,1374.0
12500,6.0,0.0,0.0,0.267857,0.000000,0.000,6.0,0.000000,0.0,0.0,...,0.000000,0.198548,0.125,0.125,0.125,0.125,0.125,0.125,0.125,1374.0


In [6]:
isv_nr = (wagepan.groupby("nr").var() == 0)
noVar_nr = isv_nr.all(axis = 0)
# noVar_nr,
isv_nr.columns[noVar_nr]

Index(['black', 'hisp', 'educ'], dtype='object')

In [7]:
isv_t = wagepan.groupby("nr").var() == 0
noVar_t = isv_t.all(axis = 0)
# isv_t, 
isv_t.columns[noVar_t]

Index(['black', 'hisp', 'educ'], dtype='object')

In [8]:
wagepan.set_index(["nr", "year"], drop = False, inplace = True)

In [9]:
formula = "lwage ~ educ + black + hisp + exper + I(exper**2) + married + union + C(year)"

reg_pooled =plm.PooledOLS.from_formula(formula = formula,
                                    data = wagepan)
results_pooled = reg_pooled.fit()
LM_summary(results_pooled)

,$\beta $,$t$,$$P>|t|$$,$SE$
$$r^2: 0.1893$$,,,,
C(year)[T.1980],0.092056,1.176130,2.396076e-01,0.078270
C(year)[T.1981],0.150376,1.793474,7.296674e-02,0.083846
C(year)[T.1982],0.154830,1.733458,8.308526e-02,0.089319
C(year)[T.1983],0.154068,1.632282,1.026926e-01,0.094388
C(year)[T.1984],0.182523,1.843739,6.528920e-02,0.098996
C(year)[T.1985],0.201302,1.952284,5.096858e-02,0.103111
C(year)[T.1986],0.234015,2.191982,2.843369e-02,0.106760
C(year)[T.1987],0.265889,2.416628,1.570597e-02,0.110025
I(exper**2),-0.002412,-2.941264,3.286019e-03,0.000820


In [10]:
# just time effects same as pooled with time effects
reg_te = plm.PanelOLS.from_formula(formula = formula,
                                    data = wagepan)
results_te =reg_te.fit()
LM_summary(results_te)

,$\beta $,$t$,$$P>|t|$$,$SE$
$$r^2: 0.1893$$,,,,
C(year)[T.1980],0.092056,1.176130,2.396076e-01,0.078270
C(year)[T.1981],0.150376,1.793474,7.296674e-02,0.083846
C(year)[T.1982],0.154830,1.733458,8.308526e-02,0.089319
C(year)[T.1983],0.154068,1.632282,1.026926e-01,0.094388
C(year)[T.1984],0.182523,1.843739,6.528920e-02,0.098996
C(year)[T.1985],0.201302,1.952284,5.096858e-02,0.103111
C(year)[T.1986],0.234015,2.191982,2.843369e-02,0.106760
C(year)[T.1987],0.265889,2.416628,1.570597e-02,0.110025
I(exper**2),-0.002412,-2.941264,3.286019e-03,0.000820


In [11]:
formula = "lwage ~ I(exper**2) + married + union + C(year)"
# Time and Entity Effects
reg_fe = plm.PanelOLS.from_formula(formula = formula + " + EntityEffects",
                                    data = wagepan)
results_fe = reg_fe.fit()
LM_summary(results_fe)

,$\beta $,$t$,$$P>|t|$$,$SE$
$$r^2: 0.1806$$,,,,
C(year)[T.1980],1.426019,77.748354,0.000000e+00,0.018341
C(year)[T.1981],1.577210,72.965647,0.000000e+00,0.021616
C(year)[T.1982],1.678989,63.258282,0.000000e+00,0.026542
C(year)[T.1983],1.780462,53.439178,0.000000e+00,0.033318
C(year)[T.1984],1.916133,45.981630,0.000000e+00,0.041672
C(year)[T.1985],2.043501,39.646046,0.000000e+00,0.051544
C(year)[T.1986],2.191515,34.771405,0.000000e+00,0.063026
C(year)[T.1987],2.351043,30.866912,0.000000e+00,0.076167
I(exper**2),-0.005185,-7.361196,2.220446e-13,0.000704


In [12]:
formula = "lwage ~ educ + black + hisp + exper + I(exper**2) + married + union + C(year)"
reg_re = plm.RandomEffects.from_formula(formula = formula,
                                    data = wagepan)

results_re = reg_re.fit()
LM_summary(results_re)

,$\beta $,$t$,$$P>|t|$$,$SE$
$$r^2: 0.1806$$,,,,
C(year)[T.1980],0.023414,0.154606,8.771388e-01,0.151441
C(year)[T.1981],0.063837,0.398786,6.900706e-01,0.160079
C(year)[T.1982],0.054265,0.321115,7.481389e-01,0.168989
C(year)[T.1983],0.043602,0.244962,8.064974e-01,0.177995
C(year)[T.1984],0.066427,0.355113,7.225219e-01,0.187057
C(year)[T.1985],0.081109,0.413565,6.792131e-01,0.196122
C(year)[T.1986],0.115241,0.561652,5.743819e-01,0.205182
C(year)[T.1987],0.158250,0.738620,4.601778e-01,0.214251
I(exper**2),-0.004729,-6.862314,7.731593e-12,0.000689


#### We can compare all results in a single table

In [13]:
from linearmodels.panel import compare

compare({"FE": results_fe, "RE": results_re, "Pooled": results_pooled})

,FE,RE,Pooled
Dep. Variable,lwage,lwage,lwage
Estimator,PanelOLS,RandomEffects,PooledOLS
No. Observations,4360,4360,4360
Cov. Est.,Unadjusted,Unadjusted,Unadjusted
R-squared,0.1806,0.1806,0.1893
R-Squared (Within),0.1806,0.1799,0.1692
R-Squared (Between),-0.0052,0.1853,0.2066
R-Squared (Overall),0.0807,0.1828,0.1893
F-statistic,83.851,68.409,72.459
P-value (F-stat),0.0000,0.0000,0.0000


### Hausman Test 
#### From Appendix: Script 14.4: Example-HausmTest.py

In [14]:
b_fe = results_fe.params
b_fe_cov = results_fe.cov
b_re = results_re.params
b_re_cov = results_re.cov

common_coef = set(results_fe.params.index).intersection(results_re.params.index)
common_coef

{'C(year)[T.1980]',
 'C(year)[T.1981]',
 'C(year)[T.1982]',
 'C(year)[T.1983]',
 'C(year)[T.1984]',
 'C(year)[T.1985]',
 'C(year)[T.1986]',
 'C(year)[T.1987]',
 'I(exper**2)',
 'married',
 'union'}

In [15]:
b_diff = results_fe.params[common_coef].sub(results_re.params[common_coef])
df = len(b_diff)
b_diff

C:\Users\JLCat\AppData\Local\Temp\ipykernel_21720\1901612778.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  b_diff = results_fe.params[common_coef].sub(results_re.params[common_coef])
C:\Users\JLCat\AppData\Local\Temp\ipykernel_21720\1901612778.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  b_diff = results_fe.params[common_coef].sub(results_re.params[common_coef])


C(year)[T.1986]    2.076274
C(year)[T.1982]    1.624725
married           -0.017134
union             -0.025865
I(exper**2)       -0.000456
C(year)[T.1983]    1.736860
C(year)[T.1980]    1.402605
C(year)[T.1981]    1.513373
C(year)[T.1987]    2.192793
C(year)[T.1985]    1.962392
C(year)[T.1984]    1.849707
Name: parameter, dtype: float64

In [16]:
b_cov_diff = b_fe_cov.loc[common_coef, common_coef].sub(b_re_cov.loc[common_coef, common_coef])
b_cov_diff

C:\Users\JLCat\AppData\Local\Temp\ipykernel_21720\1192123000.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  b_cov_diff = b_fe_cov.loc[common_coef, common_coef].sub(b_re_cov.loc[common_coef, common_coef])
C:\Users\JLCat\AppData\Local\Temp\ipykernel_21720\1192123000.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  b_cov_diff = b_fe_cov.loc[common_coef, common_coef].sub(b_re_cov.loc[common_coef, common_coef])


,C(year)[T.1986],C(year)[T.1982],married,union,I(exper**2),C(year)[T.1983],C(year)[T.1980],C(year)[T.1981],C(year)[T.1987],C(year)[T.1985],C(year)[T.1984]
C(year)[T.1986],-0.038127,-0.032788,-4.116160e-04,-1.188561e-04,-6.415442e-05,-0.034300,-0.029378,-0.031141,-0.039140,-0.036976,-0.035698
C(year)[T.1982],-0.032788,-0.027853,-2.700965e-04,-8.933812e-05,-3.376926e-05,-0.029178,-0.024998,-0.026455,-0.033852,-0.031650,-0.030444
married,-0.000412,-0.000270,5.377188e-05,7.736228e-07,1.858870e-07,-0.000307,-0.000198,-0.000235,-0.000447,-0.000377,-0.000342
union,-0.000119,-0.000089,7.736228e-07,5.404630e-05,-9.915058e-08,-0.000097,-0.000072,-0.000080,-0.000129,-0.000112,-0.000105
I(exper**2),-0.000064,-0.000034,1.858870e-07,-9.915058e-08,2.128958e-08,-0.000041,-0.000019,-0.000026,-0.000072,-0.000056,-0.000049
C(year)[T.1983],-0.034300,-0.029178,-3.068184e-04,-9.706718e-05,-4.130821e-05,-0.030572,-0.026143,-0.027697,-0.035373,-0.033137,-0.031892
C(year)[T.1980],-0.029378,-0.024998,-1.981415e-04,-7.180325e-05,-1.882321e-05,-0.026143,-0.022598,-0.023814,-0.030377,-0.028336,-0.027256
C(year)[T.1981],-0.031141,-0.026455,-2.348176e-04,-8.043469e-05,-2.627847e-05,-0.027697,-0.023814,-0.025158,-0.032180,-0.030043,-0.028894
C(year)[T.1987],-0.039140,-0.033852,-4.471107e-04,-1.291239e-04,-7.184795e-05,-0.035373,-0.030377,-0.032180,-0.040102,-0.038024,-0.036766
C(year)[T.1985],-0.036976,-0.031650,-3.767402e-04,-1.118637e-04,-5.649780e-05,-0.033137,-0.028336,-0.030043,-0.038024,-0.035807,-0.034524


In [17]:
b_var_diff = pd.Series(np.diag(b_cov_diff), index=[b_cov_diff.index])
b_var_diff

C(year)[T.1986]   -3.812734e-02
C(year)[T.1982]   -2.785277e-02
married            5.377188e-05
union              5.404630e-05
I(exper**2)        2.128958e-08
C(year)[T.1983]   -3.057219e-02
C(year)[T.1980]   -2.259812e-02
C(year)[T.1981]   -2.515791e-02
C(year)[T.1987]   -4.010217e-02
C(year)[T.1985]   -3.580718e-02
C(year)[T.1984]   -3.325389e-02
dtype: float64

In [18]:
stat = abs(b_diff.T @ np.linalg.inv(b_cov_diff) @ b_diff)
p_val = 1 - stats.chi2.cdf(stat, df)
stat, p_val

(43.427071175957124, 9.150613850206213e-06)

In [19]:
def hausman_test(results_fe, results_re):
    b_fe = results_fe.params
    b_fe_cov = results_fe.cov
    b_re = results_re.params
    b_re_cov = results_re.cov
    common_coef = set(results_fe.params.index).intersection(results_re.params.index)    
    b_diff = results_fe.params[common_coef].sub(results_re.params[common_coef])
    df = len(b_diff)
    b_cov_diff = b_fe_cov.loc[common_coef, common_coef].sub(b_re_cov.loc[common_coef, common_coef])
    b_var_diff = pd.Series(np.diag(b_cov_diff), index=[b_cov_diff.index])
    h_results = {"Hausman":{}}
    h_results["Hausman"]["t"] = abs(b_diff.T @ np.linalg.inv(b_cov_diff) @ b_diff)
    h_results["Hausman"]["p"] = 1 - stats.chi2.cdf(stat, df)
    return pd.DataFrame(h_results)
hausman_test(results_fe, results_re).round(3)

C:\Users\JLCat\AppData\Local\Temp\ipykernel_21720\3980050239.py:7: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  b_diff = results_fe.params[common_coef].sub(results_re.params[common_coef])
C:\Users\JLCat\AppData\Local\Temp\ipykernel_21720\3980050239.py:7: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  b_diff = results_fe.params[common_coef].sub(results_re.params[common_coef])
C:\Users\JLCat\AppData\Local\Temp\ipykernel_21720\3980050239.py:9: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  b_cov_diff = b_fe_cov.loc[common_coef, common_coef].sub(b_re_cov.loc[common_coef, common_coef])
C:\Users\JLCat\AppData\Local\Temp\ipykernel_21720\3980050239.py:9: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  b_cov_diff = b_fe_cov.lo

,Hausman
p,0.000
t,43.427


It turns out that we can get the FE parameter estimates in two other ways than the within transformation we used in Section 14.1. The dummy variable regression uses OLS on the original variables in Equation 13.2 instead of the transformed ones. But it adds n - 1 dummy variables.

$$a_i = \gamma_0 + \gamma_1 \bar{x}_{i1} + . . . + \gamma \bar{x}_{ik} + r_i$$

$$y_{it} = \beta_0 + \beta_1 x_{it1} + . . . + \beta_kx_{itk} + a_i + u_{it}$$


In [20]:
wagepan["entity"] = wagepan["nr"]
wagepan.set_index("nr", inplace = True)#, drop = False)
wagepan["married_b"] = wagepan.groupby("nr").mean()["married"]
wagepan["union_b"] = wagepan.groupby("nr").mean()["union"]
wagepan = wagepan.set_index(["year"], append = True, drop = False)
wagepan

year  agric  black  bus  construc  ent  exper  fin  hisp  \
nr    year                                                             
13    1980  1980      0      0    1         0    0      1    0     0   
      1981  1981      0      0    0         0    0      2    0     0   
      1982  1982      0      0    1         0    0      3    0     0   
      1983  1983      0      0    1         0    0      4    0     0   
      1984  1984      0      0    0         0    0      5    0     0   
...          ...    ...    ...  ...       ...  ...    ...  ...   ...   
12548 1983  1983      0      0    0         1    0      8    0     0   
      1984  1984      0      0    0         1    0      9    0     0   
      1985  1985      0      0    0         1    0     10    0     0   
      1986  1986      0      0    0         0    0     11    0     0   
      1987  1987      0      0    0         0    0     12    0     0   

            poorhlth  ...  d82  d83  d84  d85  d86  d87  expersq  entity  \
nr    year            ...                                                  
13    1980         0  ...    0    0    0    0    0    0        1      13   
      1981         0  ...    0    0    0    0    0    0        4      13   
      1982         0  ...    1    0    0    0    0    0        9      13   
      1983         0  ...    0    1    0    0    0    0       16      13   
      1984         0  ...    0    0    1    0    0    0       25      13   
...              ...  ...  ...  ...  ...  ...  ...  ...      ...     ...   
12548 1983         0  ...    0    1    0    0    0    0       64   12548   
      1984         0  ...    0    0    1    0    0    0       81   12548   
      1985         0  ...    0    0    0    1    0    0      100   12548   
      1986         0  ...    0    0    0    0    1    0      121   12548   
      1987         0  ...    0    0    0    0    0    1      144   12548   

            married_b  union_b  
nr    year                      
13    1980      0.000    0.125  
      1981      0.000    0.125  
      1982      0.000    0.125  
      1983      0.000    0.125  
      1984      0.000    0.125  
...               ...      ...  
12548 1983      0.625    0.375  
      1984      0.625    0.375  
      1985      0.625    0.375  
      1986      0.625    0.375  
      1987      0.625    0.375  

[4360 rows x 46 columns]

In [21]:
formula = "lwage ~ married + union + C(year)*educ + EntityEffects"
reg_we = plm.PanelOLS.from_formula(formula = formula,
                                  drop_absorbed = True, data = wagepan)
results_we = reg_we.fit()
LM_summary(results_we)

C:\Users\JLCat\AppData\Local\Temp\ipykernel_21720\185446945.py:4: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

educ

  results_we = reg_we.fit()


,$\beta $,$t$,$$P>|t|$$,$SE$
$$r^2: 0.1708$$,,,,
C(year)[T.1980],1.362459,83.903110,0.000000,0.016238
C(year)[T.1981],1.340043,9.230656,0.000000,0.145173
C(year)[T.1982],1.356698,9.348067,0.000000,0.145131
C(year)[T.1983],1.372888,9.456084,0.000000,0.145186
C(year)[T.1984],1.446833,9.961677,0.000000,0.145240
C(year)[T.1985],1.412184,9.731511,0.000000,0.145115
C(year)[T.1986],1.428065,9.840444,0.000000,0.145122
C(year)[T.1987],1.452904,10.006096,0.000000,0.145202
married,0.054820,2.977340,0.002926,0.018413


In [22]:
formula = "lwage ~ married + union + C(year)*educ + C(entity)"
reg_dum = smf.ols(formula = formula,
                  data = wagepan)
results_dum = reg_dum.fit()
OLS_summary(results_dum)

,$\beta $,$t$,$$P>|t|$$,$SE$
$$r^2: 0.6164$$,,,,
Intercept,2.1080,5.8568,0.0000,0.3599
C(year)[T.1981],-0.0224,-0.1537,0.8779,0.1459
C(year)[T.1982],-0.0058,-0.0395,0.9685,0.1459
C(year)[T.1983],0.0104,0.0715,0.9430,0.1459
C(year)[T.1984],0.0844,0.5785,0.5630,0.1459
...,...,...,...,...
C(year)[T.1983]:educ,0.0171,1.3959,0.1628,0.0123
C(year)[T.1984]:educ,0.0166,1.3521,0.1764,0.0123
C(year)[T.1985]:educ,0.0237,1.9316,0.0535,0.0123


In [23]:
formula = "lwage ~ married + union + C(year)*educ + married_b + union_b"
reg_cre = plm.RandomEffects.from_formula(formula = formula,
                                  data = wagepan)
results_cre = reg_cre.fit()
LM_summary(results_cre)

,$\beta $,$t$,$$P>|t|$$,$SE$
$$r^2: 0.1739$$,,,,
C(year)[T.1980],0.577921,4.043669,5.353429e-05,0.142920
C(year)[T.1981],0.555506,3.886793,1.030935e-04,0.142921
C(year)[T.1982],0.572160,4.003608,6.342595e-05,0.142911
C(year)[T.1983],0.588351,4.116912,3.911186e-05,0.142911
C(year)[T.1984],0.662296,4.633842,3.694655e-06,0.142926
C(year)[T.1985],0.627647,4.391740,1.151257e-05,0.142915
C(year)[T.1986],0.643528,4.502529,6.891094e-06,0.142926
C(year)[T.1987],0.668366,4.676713,3.003680e-06,0.142914
educ,0.058594,4.917946,9.069083e-07,0.011914


In [27]:
formula = "lwage ~ married + union + C(year)*educ"
reg_re = plm.RandomEffects.from_formula(formula = formula,
                                  data = wagepan)
results_re = reg_re.fit()
LM_summary(results_re)

,$\beta $,$t$,$$P>|t|$$,$SE$
$$r^2: 0.1702$$,,,,
C(year)[T.1980],0.652315,4.591780,4.519133e-06,0.142061
C(year)[T.1981],0.630936,4.441247,9.166290e-06,0.142063
C(year)[T.1982],0.648583,4.567148,5.081008e-06,0.142011
C(year)[T.1983],0.662299,4.661373,3.235258e-06,0.142082
C(year)[T.1984],0.733996,5.163699,2.529540e-07,0.142145
C(year)[T.1985],0.705038,4.965526,7.114465e-07,0.141987
C(year)[T.1986],0.721497,5.081196,3.908216e-07,0.141994
C(year)[T.1987],0.741491,5.218032,1.892690e-07,0.142102
educ,0.059473,4.987051,6.370088e-07,0.011926


In [26]:
compare({"WE": results_we,  "CRE": results_cre})#, "Pooled": results_pooled})

,WE,CRE
Dep. Variable,lwage,lwage
Estimator,PanelOLS,RandomEffects
No. Observations,4360,4360
Cov. Est.,Unadjusted,Unadjusted
R-squared,0.1708,0.1739
R-Squared (Within),0.1708,0.1708
R-Squared (Between),0.0905,0.1956
R-Squared (Overall),0.1277,0.1841
F-statistic,48.907,48.081
P-value (F-stat),0.0000,0.0000


In [32]:
names = ["married", "union", "C(year)[T.1982]:educ"]
pd.DataFrame({"Within":results_we.params[names],
              "OLS":results_dum.params[names],
              "CRE":results_cre.params[names],
              "RE":results_re.params[names]}).round(4)

,Within,OLS,CRE,RE
married,0.0548,0.0548,0.0548,0.0773
union,0.0830,0.0830,0.0830,0.1075
C(year)[T.1982]:educ,0.0148,0.0148,0.0148,0.0143


In [33]:
# Wald test using correlated random effects
wtest = results_cre.wald_test(formula="married_b = union_b = 0")
wtest

Linear Equality Hypothesis Test
H0: Linear equality constraint is valid
Statistic: 19.4058
P-value: 0.0001
Distributed: chi2(2)
WaldTestStatistic, id: 0x20839e63c10